# Importing Necessary Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()

In [3]:
raw = pd.read_csv('sinofi_raw.csv')

In [4]:
raw.head()

,product_code,product_type,customer_code,market_code,market_name,order_date,sales_qty,upc,sales_amount,zone
0,Prod001,Syrup,Cus001,Mark001,Darbhanga,10-10-2017,100,272.1840,41241,Bihar
1,Prod001,Syrup,Cus002,Mark002,Patna,08-05-2018,3,272.1840,-1,Bihar
2,Prod002,Ointment,Cus003,Mark003,Bhagalpur,06-04-2018,1,37.9896,875,Bihar
3,Prod002,Ointment,Cus003,Mark003,Bhagalpur,11-04-2018,1,37.9896,583,Bihar
4,Prod002,Ointment,Cus004,Mark003,Bhagalpur,18-06-2018,6,37.9896,7176,Bihar


# Data Cleaning

### Checking null values in the dataframe

In [5]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150283 entries, 0 to 150282
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   product_code   150283 non-null  object 
 1   product_type   150283 non-null  object 
 2   customer_code  150283 non-null  object 
 3   market_code    150283 non-null  object 
 4   market_name    150283 non-null  object 
 5   order_date     150283 non-null  object 
 6   sales_qty      150283 non-null  int64  
 7   upc            150283 non-null  float64
 8   sales_amount   150283 non-null  int64  
 9   zone           150283 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 11.5+ MB


In [6]:
raw.count().isnull()

product_code     False
product_type     False
customer_code    False
market_code      False
market_name      False
order_date       False
sales_qty        False
upc              False
sales_amount     False
zone             False
dtype: bool

There is no null vales in any column

### Checking duplicate entries

In [7]:
len(raw)

150283

In [8]:
# Number of duplicate rows

raw.duplicated().value_counts()

False    150004
True        279
dtype: int64

In [9]:
len(raw)-len(raw.drop_duplicates())

279

There are 279 duplicate rows. We will remove it.

In [10]:
# Remove duplicate rows
raw.drop_duplicates(inplace=True)

In [11]:
# Check if duplicates has been removed or not

len(raw)-len(raw.drop_duplicates())

0

In [12]:
raw.duplicated().value_counts()

False    150004
dtype: int64

### Treating zero and negative sales_amount

In [13]:
raw.describe()

,sales_qty,upc,sales_amount
count,150004.000000,150004.000000,1.500040e+05
mean,16.280579,86.674874,6.565648e+03
std,114.859234,186.126662,2.993264e+04
min,1.000000,0.000000,-1.000000e+00
25%,1.000000,11.187116,1.670000e+02
50%,1.000000,37.911939,5.050000e+02
75%,7.000000,98.284074,3.000000e+03
max,14049.000000,19501.600800,1.510944e+06


The minimum sales_amount has a negative value which is practically not possible, even a zero is not possible. This error might have happened due to human error while filling in the data. The negative values needs to be treated but, before that lets see which columns have zero values in them.

In [14]:
# Checking columns with zero values.

(raw.iloc[:,0:raw.shape[1]]==0).sum()

product_code        0
product_type        0
customer_code       0
market_code         0
market_name         0
order_date          0
sales_qty           0
upc                 1
sales_amount     1606
zone                0
dtype: int64

The column upc has 1 and sales amount has 1606 zero values. First we will treat sales_amount. 

There are 1606 rows where sales_amount is zero. It con not be treated statistically as it depends on many features. It will be removed instead.

In [15]:
raw.drop(raw[raw.sales_amount ==0].index,inplace=True)

In [16]:
# Checking check if zero sales_amount rows are remover or not. 

len(raw[raw.sales_amount==0])

0

The sales_amount column has now no zero values. Now let's inspect upc column with zero value.

In [17]:
raw[raw.upc==0].count()

product_code     0
product_type     0
customer_code    0
market_code      0
market_name      0
order_date       0
sales_qty        0
upc              0
sales_amount     0
zone             0
dtype: int64

It seems that the row which has zero upc value was removed while removing rows with zero sales_amount.

### Treating negative values

In [18]:
raw.describe()

,sales_qty,upc,sales_amount
count,148398.000000,148398.000000,1.483980e+05
mean,16.370706,86.311603,6.636703e+03
std,115.393172,186.204156,3.008634e+04
min,1.000000,0.858000,-1.000000e+00
25%,1.000000,10.549199,1.760000e+02
50%,1.000000,37.911939,5.190000e+02
75%,7.000000,98.223714,3.065000e+03
max,14049.000000,19501.600800,1.510944e+06


1. The sales_amount is the only column which has a negative minimum value, it means that no other numerical columns have a negative value.
2. The minimum sales amount is negative which is practically not possible. This error might have happened due to human error.

In [19]:
# sales amount rows with negative values

raw[raw.sales_amount<0]

,product_code,product_type,customer_code,market_code,market_name,order_date,sales_qty,upc,sales_amount,zone
1,Prod001,Syrup,Cus002,Mark002,Patna,08-05-2018,3,272.184,-1,Bihar


There is one row with negative sales_amount.

If there would have been many rows with negative sales_amount then we woud have treated them statistically but here we will simply remove that row.

In [20]:
# Drop Row with negative sales_amount

raw.drop(raw[raw.sales_amount < 0].index, inplace=True)

In [21]:
# Check if row with negative sales_amount has been removed or not
# Method 1

raw[raw.sales_amount<0]

,product_code,product_type,customer_code,market_code,market_name,order_date,sales_qty,upc,sales_amount,zone


In [22]:
# Check if row with negative sales_amount has been removed or not
# Method 2

raw[raw.sales_amount<0].count()

product_code     0
product_type     0
customer_code    0
market_code      0
market_name      0
order_date       0
sales_qty        0
upc              0
sales_amount     0
zone             0
dtype: int64

Row with negative sales_amount has been removed 

In [23]:
len(raw)

148397

After all the necessary data processing 148397 rows are left.

In [24]:
raw.describe()

,sales_qty,upc,sales_amount
count,148397.000000,148397.000000,1.483970e+05
mean,16.370796,86.310351,6.636748e+03
std,115.393556,186.204158,3.008643e+04
min,1.000000,0.858000,5.000000e+00
25%,1.000000,10.549199,1.760000e+02
50%,1.000000,37.911939,5.190000e+02
75%,7.000000,98.223714,3.065000e+03
max,14049.000000,19501.600800,1.510944e+06


### Feature Engineering

1. Add net profit column.
2. Net profit = sales_amount - (sales_qty * UPC)

In [25]:
raw['net_profit'] = raw.sales_amount - (raw.upc*raw.sales_qty)
raw.head()

,product_code,product_type,customer_code,market_code,market_name,order_date,sales_qty,upc,sales_amount,zone,net_profit
0,Prod001,Syrup,Cus001,Mark001,Darbhanga,10-10-2017,100,272.1840,41241,Bihar,14022.6000
2,Prod002,Ointment,Cus003,Mark003,Bhagalpur,06-04-2018,1,37.9896,875,Bihar,837.0104
3,Prod002,Ointment,Cus003,Mark003,Bhagalpur,11-04-2018,1,37.9896,583,Bihar,545.0104
4,Prod002,Ointment,Cus004,Mark003,Bhagalpur,18-06-2018,6,37.9896,7176,Bihar,6948.0624
5,Prod003,Tablet,Cus005,Mark004,Muzaffarpur,20-11-2017,59,646.6317,40000,Bihar,1848.7297


In [26]:
# Export the current table 'raw'

raw.to_csv (r'C:\Users\atifn\Downloads\sinofi_raw.csv', index = False, header=True)

### Data Fragmentation

#### 1. Making customer table 

In [27]:
# Making customers dataframe
customers = raw[['customer_code','market_code','market_name','zone']].drop_duplicates('customer_code')

# sort in ascending customer_code 
customers.sort_values(by='customer_code',ascending=True).head()

,customer_code,market_code,market_name,zone
0,Cus001,Mark001,Darbhanga,Bihar
573,Cus002,Mark002,Patna,Bihar
2,Cus003,Mark003,Bhagalpur,Bihar
4,Cus004,Mark003,Bhagalpur,Bihar
5,Cus005,Mark004,Muzaffarpur,Bihar


In [28]:
# Download customer table on desired location.

customers.sort_values(by='customer_code',ascending=True).to_csv (r'C:\Users\atifn\Downloads\customers.csv', index = False, header=True)

#### 2. Making markets table

In [29]:
# Making markets dataframe
markets = raw[['market_code','market_name','zone']].drop_duplicates('market_code')

# Sort in ascending market_code 
markets.sort_values(by='market_code', ascending=True).head()

,market_code,market_name,zone
0,Mark001,Darbhanga,Bihar
123,Mark002,Patna,Bihar
2,Mark003,Bhagalpur,Bihar
5,Mark004,Muzaffarpur,Bihar
116,Mark005,Siliguri,West_Bengal


In [30]:
# Download markets table on desired location.

markets.sort_values(by='market_code',ascending=True).to_csv (r'C:\Users\atifn\Downloads\markets.csv', index = False, header=True)

#### 3. Making products table

In [31]:
# Making products dataframe
products = raw[['product_code','product_type']].drop_duplicates('product_code')

# Sort in ascending product_code 
products.sort_values(by='product_code',ascending=True).head()

,product_code,product_type
0,Prod001,Syrup
2,Prod002,Ointment
5,Prod003,Tablet
12,Prod004,Syrup
14,Prod005,Tablet


In [32]:
# Download products table

products.sort_values(by='product_code',ascending=True).to_csv(r'C:\Users\atifn\Downloads\products.csv',index=False,header=True)

#### 4. Making upc table

In [33]:
# Making upc dataframe
upc = raw[['product_code','upc']].drop_duplicates('product_code')

# Sort in ascending product_code 
upc.sort_values(by='product_code',ascending=True).head()

,product_code,upc
0,Prod001,272.184000
2,Prod002,37.989600
5,Prod003,646.631700
12,Prod004,17.955600
14,Prod005,108.441086


In [34]:
# Download upc table

upc.sort_values(by='product_code',ascending=True).to_csv(r'C:\Users\atifn\Downloads\upc.csv',index=False,header=True)

#### 5. Making transaction table
Transaction table would be the raw table after data cleaning.

In [35]:
transaction = raw
transaction.head()

,product_code,product_type,customer_code,market_code,market_name,order_date,sales_qty,upc,sales_amount,zone,net_profit
0,Prod001,Syrup,Cus001,Mark001,Darbhanga,10-10-2017,100,272.1840,41241,Bihar,14022.6000
2,Prod002,Ointment,Cus003,Mark003,Bhagalpur,06-04-2018,1,37.9896,875,Bihar,837.0104
3,Prod002,Ointment,Cus003,Mark003,Bhagalpur,11-04-2018,1,37.9896,583,Bihar,545.0104
4,Prod002,Ointment,Cus004,Mark003,Bhagalpur,18-06-2018,6,37.9896,7176,Bihar,6948.0624
5,Prod003,Tablet,Cus005,Mark004,Muzaffarpur,20-11-2017,59,646.6317,40000,Bihar,1848.7297


In [36]:
# Download transaction table

transaction.to_csv(r'C:\Users\atifn\Downloads\transaction.csv',index=False,header=True)